In [1]:
import pandas as pd
df = pd.read_csv('../data/Original_csv/Hyperion.csv')

In [2]:
df.head(5)

,Domanda,Testo,Stralcio,Repertorio,Contesto
0,A cosa serve questo testo per il perseguimento...,Dunque vediamo se ho capito: Conte le canta a ...,Dunque vediamo se ho capito:,dichiarazione di intenti,Dunque vediamo se ho capito: Conte le canta a ...
1,NaN,NaN,Conte le canta a Salvini e alla Meloni,sancire,Dunque vediamo se ho capito: questi reagiscon...
2,NaN,NaN,questi reagiscono e le cantano a loro volta a ...,giustificazione,Conte le canta a Salvini e alla Meloni Mentan...
3,NaN,NaN,Mentana si smarca da Conte e finisce con foto ...,sancire,questi reagiscono e le cantano a loro volta a ...
4,NaN,NaN,Siamo tornati per un attimo alla normalità:,commento,Mentana si smarca da Conte e finisce con foto ...


In [4]:
import string

def find_word_bounds(spans: list, text: str) -> list:
    bounds = []
    end = 0
    for span in spans:
        s = span.translate(str.maketrans('', '', string.punctuation))
        word_list = s.split()
        text_list = text.translate(str.maketrans('', '', string.punctuation)).split()
        try:
            start = text_list.index(word_list[0], end)
        except:
            if not bounds:
                start = 0
            else:
                
                start = bounds[-1][1] + 1
        end = start + len(word_list) - 1
            
        bounds.append((start, end))
    return bounds

def IE_gen(bounds:list) -> str:
    end = [bound[1] for bound in bounds]
    x = ['E' if i in end else 'I' for i in range(end[-1]+1)]
    return ''.join(x)

In [5]:
dataset = []

for row in df.itertuples():
    text = row.Testo
    if pd.isna(text):
        sample['Stralci'].append(row.Stralcio)
    else: 
        sample = {}
        sample['Testo'] = text
        sample['Stralci'] = [row.Stralcio]
        dataset.append(sample)
        



for sample in dataset:
    sample['Bounds'] = find_word_bounds(sample['Stralci'], sample['Testo'])
    sample['Tags'] = IE_gen(sample['Bounds'])


In [6]:
IE_dict = {
    'Testo' : [sample['Testo'] for sample in dataset],
    'Tags' : [sample['Tags'] for sample in dataset] 
}
IE_df = pd.DataFrame(IE_dict)

In [7]:
IE_df.head(5)

,Testo,Tags
0,Dunque vediamo se ho capito: Conte le canta a ...,IIIIEIIIIIIIEIIIIIIIIIEIIIIIIIIIIIIIIEIIIIIIEIE
1,#Conte è un pericolo per la libertà e la democ...,IIIIIIIIIE
2,"Comunque, se tutti i giornalisti che oggi si s...",EIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIEIIIIE
3,"""Ha suscitato scandalo la proposta di Salvini ...",IIIIIIIIIIIIEIIIIIIIIIIIIIIIIIIIIEIIIIIIIIIIII...
4,Non era mai successo nella lunga storia del Pi...,IIIIIIIIIIIIIIIIIIIIIEIIIIIIIIIIIIIEIIEIIIIIII...


In [106]:
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn import preprocessing
import numpy as np
import torch

LABELS = [
                'I',
                'E'
        ]

def encode_labels(labels_list):
    le = preprocessing.LabelEncoder()
    le.fit(LABELS)
    return le.transform(labels_list)

def first_token_label_econding(sample, labels):
    encoded_labels = np.ones(len(sample["offset_mapping"]), dtype=int) * -100
    # set only labels whose first offset position is 0 and the second is not 0
    i = 0
    for idx, mapping in enumerate(sample["offset_mapping"]):
        if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1


class IE_Hyperion_dataset(Dataset):
    def __init__(self, df, tokenizer_name):
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.df = df

    def __getitem__(self, idx):
        text = self.df['Testo'].iloc[idx]
        encoding = self.tokenizer(text,
                            #is_pretokenized=True, 
                            return_special_tokens_mask=True, 
                            add_special_tokens=True,
                            return_attention_mask=True,
                            padding='max_length',
                            truncation=True,
                        )
        labels = encode_labels(list(self.df['Tags'].iloc[idx]))
        encoded_labels = np.ones(len(encoding['input_ids']), dtype=int) * -100
        
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        #print(encoding["offset_mapping"])
        for idx, e in enumerate(encoding['special_tokens_mask']):
            token = self.tokenizer.convert_ids_to_tokens(encoding['input_ids'][idx])
            punc = token.translate(str.maketrans('', '', string.punctuation)) #empty if the token ias only punctuation
            if e == 0 and not '##' in token and  punc :
                # overwrite label
                encoded_labels[idx] = labels[i]
                i += 1
        
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        return item


    def __len__(self):
        return self.len  


In [107]:
model_name = "bert-base-multilingual-cased"
train_size = 0.8
train_df = IE_df.sample(frac=train_size,random_state=200)
test_df = IE_df.drop(train_df.index).reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

print("FULL Dataset: {}".format(IE_df.shape))
print("TRAIN Dataset: {}".format(train_df.shape))
print("TEST Dataset: {}".format(test_df.shape))

train_dataset = IE_Hyperion_dataset(IE_df, model_name) 
test_dataset = IE_Hyperion_dataset(IE_df, model_name) 

FULL Dataset: (15332, 2)
TRAIN Dataset: (12266, 2)
TEST Dataset: (3066, 2)


In [105]:
train_dataset[0]

{'input_ids': tensor([   101,  14925,  56606,  11054, 109127,  10126,  13173,  13337,  12390,
            131,  49082,  10141,  57733,    169,  64831,  60906,    173,  10512,
          63372,  10342,    117,  18160,  11639,  27788,  51530,    173,  10141,
          57733,  10343,    169,  12321,  13085,    169,  49082,    117,  13026,
          35526,  10294,  39709,  10354,  10425,  10143,  49082,    173,  59222,
          24176,  10173,  27218,  42892,  17403,  13063, 100603,  12142,  10286,
          11730,  10280,    119,  75294,  10133,  24553,  10325,  10178,  10119,
          77985,  11033,  10512,  16626,  15451,    131,  71959,  36534,  13217,
          99038,    106,    102,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0

In [108]:
for token, label in zip(train_dataset.tokenizer.convert_ids_to_tokens(train_dataset[0]["input_ids"]), train_dataset[0]["labels"]):
  print('{0:10}  {1}'.format(token, label))

IndexError: index 47 is out of bounds for axis 0 with size 47